In [127]:
import requests
import lxml
import sys
from bs4 import BeautifulSoup
import xlwt
import time
import urllib

import pandas as pd
import numpy as np

import re

In [128]:
# 引入 “抓取所有数据.py.ipynb” 里面的所有数据

df = pd.read_excel ('data/result2.xls')

gd = df.股东

,公司名字,原本表格里面的文字,企查查搜索结果数,公司标签,法定法人,注册资本,成立日期,法人邮箱,法人电话,公司地址,公司状态,股东,网站
0,七台河市利丰源商贸有限公司,七台河利丰源商贸有限公司,1,,徐庆利,30万元人民币,2016-09-26,电话,\n 邮箱：\n ...,黑龙江省七台河市桃山区桃南街怡安小区7号楼12号车库,存续,徐庆利; 赵施梅;,https://www.qcc.com//firm/64EC397.shtml
1,三亚凤凰惠隆冷饮商行,三亚凤凰惠隆冷饮商行,1,,郭伟弟,-,2012-11-28,电话,\n 邮箱：\n ...,海南省三亚市天涯区芒果村,存续,陈南光;,https://www.qcc.com//firm/87TY0Z2.shtml
2,三明市合林贸易有限公司,三明合林贸易有限公司,4,,吴合林,50万元人民币,2019-11-20,电话,\n 邮箱：\n ...,福建省三明市三元区富文路11号6幢202、203室,存续,蔡琦; 吴合林;,https://www.qcc.com//firm/830GLN7.shtml
3,三河市俊鑫奕城商贸有限公司,三河市俊鑫奕城商贸有限公司,1,,李春梅,200万元人民币,2015-01-13,电话,\n 邮箱：\n ...,河北省廊坊市三河市燕郊高新区南外环路北侧金谷·爱舒荷第A10幢1单元14层1406号房,存续,李春梅; 李俊; 吴艳丽;,https://www.qcc.com//firm/69SUPSP.shtml
4,上海久耶供应链管理有限公司,上海久耶供应链管理有限公司,19,战略投资,张冰,333.7607万元人民币,2014-11-10,电话,\n 邮箱：\n ...,官网：\n www.jiuy...,存续,张冰; 峰尚达资本投资管理（深圳）有限公司; 张乐; 侯高阳; 上海郑明投资（集团）有限公司...,https://www.qcc.com//firm/94QA4S3.shtml
...,...,...,...,...,...,...,...,...,...,...,...,...,...
482,宝鸡尚优千品商贸有限公司,阜阳市韩雪商贸有限公司,1,,尚斌,50万元人民币,2018-12-06,电话,\n 邮箱：\n ...,陕西省宝鸡市渭滨区相家庄101号九华龙石小区东侧库房,在业,徐旭; 尚斌;,https://www.qcc.com//firm/56UY7D2.shtml
483,鸡西市易和商贸有限公司,阳谷县海洋商贸有限公司,1,,李凤莲,10万元人民币,2015-01-16,电话,\n 邮箱：\n ...,黑龙江省鸡西市鸡冠区东山阳光家园11号楼1单元103,存续,李凤莲; 李雅斌;,https://www.qcc.com//firm/62HNCQR.shtml
484,鹤岗市悦生活商贸有限公司,阳谷金泰丰商贸有限公司,1,,裴子同,30万元人民币,2016-11-14,电话,\n 邮箱：\n ...,鹤岗市南山区33委交警队2号住宅楼-105室(车库),存续,裴子同; 杨洪振;,https://www.qcc.com//firm/29K95KO.shtml
485,齐河县旭泽商贸有限公司,陕西凡佳商贸有限公司,1,,朱芬芬,60万元人民币,2018-03-01,电话,\n 邮箱：\n ...,山东省德州市齐河县城区齐贸大街392号2排4号,在业,朱芬芬; 张正婷;,https://www.qcc.com//firm/35HGRAZ.shtml


In [129]:

df_start = pd.read_excel ('data/经销商+交易对手方删了最初的.xlsx')

df_start

,Client,Type,Problem,人名
0,七台河利丰源商贸有限公司,经销商,NaN,NaN
1,三亚凤凰惠隆冷饮商行,经销商,NaN,NaN
2,三明合林贸易有限公司,经销商,NaN,NaN
3,三河市俊鑫奕城商贸有限公司,经销商,NaN,NaN
4,上海久耶供应链管理有限公司,对手方,NaN,NaN
...,...,...,...,...
510,鸡尚优千品商贸有限公司,对手方,NaN,NaN
511,鸡西市易和商贸有限公司,经销商,NaN,NaN
512,鹤岗市悦生活商贸有限公司,经销商,NaN,NaN
513,齐河县旭泽商贸有限公司,经销商,NaN,NaN


In [130]:
# 这些是企查查里面没有数据的

no_result = ['上海讯誉食品有限公司', '东光友媛商贸有限公司', '丽水福顺商贸有限公司', '义乌市奋钧贸易有限公司', '乌兰浩特市顺和食品商行', '伊川县征鸿副食商行', '北京京鑫旺旺乳品经销中心', '华东大润发欧尚系统直营', '南昌市瀚诺贸易有限公司', '商丘地区虞城县隆康食品经营部', '天津市益多邦商贸有限公司', '天津市翰宏伟业商贸有限公司', '天津忆牛商贸有限公司', '天津河东区姜学伍', '天津通劲商贸有限公司', '山东欧莱徳仪器有限公司', '峰峰矿区彭城君胜食品店', '广州淮念电子商务有限公司', '成都市京蜀商贸有限公司吧', '新宁县亚玲酸奶店', '新郑市光明副食经营部', '无锡乐事多商贸有限公司', '杭州脉兴机械有限公司', '永康市溢海商贸有限购公司', '沈阳君伊商贸有限公司秦皇岛分公司', '湖南省邵东市南际冷饮批发部', '滨州鹏仔商贸有限公司', '邯郸市永年区名关釜阳乳品门市']

len(no_result)

28

In [131]:
name_checked_lst = []

for i in range (df_start.shape[0]):
    temp = df_start.loc[i,'Client']
    
    if not temp in no_result:
        name_checked_lst.append(temp)

name_checked_lst        

['七台河利丰源商贸有限公司',
 '三亚凤凰惠隆冷饮商行',
 '三明合林贸易有限公司',
 '三河市俊鑫奕城商贸有限公司',
 '上海久耶供应链管理有限公司',
 '上海光明乳业国际贸易有限公司',
 '上海务强食品有限公司',
 '上海如天包装设备有限公司',
 '上海快行天下供应链管理有限公司',
 '上海明酷商贸有限公司',
 '上海昶进贸易有限公司',
 '上海欣融食品原料有限公司',
 '上海炫化贸易有限公司',
 '上海聚恒贸易有限公司',
 '上海良骥贸易有限公司',
 '上海语莱机械设备有限公司',
 '上海贺韬贸易有限公司',
 '上海远安流体设备科技有限公司',
 '东台市乐好商贸有限公司',
 '东海县腾宇食品销售有限公司',
 '东海县荣来商贸有限公司',
 '东莞市丰尚食品贸易有限公司',
 '东营市丽涵商贸有限公司',
 '东营市顺翔商贸有限公司',
 '东营汇尊商贸有限公司',
 '东阳市恒润商贸有限公司',
 '东阳市海纳食品有限公司',
 '东阿县昊鑫商贸有限公司',
 '丽江市古城区百优商贸有限公司',
 '中山市华哲副食商行',
 '中海实力贸易有限责任公司',
 '丰宁满族自治县宝诚商贸有限公司',
 '临沂俊辰商贸有限公司',
 '临沂市兰山区云东家食品馆',
 '临沂豪联塑业有限公司',
 '临沂靖凯欣商贸有限公司',
 '临泉县小君商贸有限公司',
 '丹阳市玖通食品商贸有限公司',
 '乌兰浩特市顺和商贸有限责任公司',
 '乌鲁木齐鼎信众诚供应链管理有限公司',
 '乐亭县泽翔商贸有限公司',
 '乐陵丰盛食品经营部',
 '云南三桥商贸有限公司',
 '云南萨普托商贸有限公司',
 '五河县青隆商贸有限公司',
 '亳州银岭机械设备有限公司',
 '仁寿盈兴旺商贸有限公司',
 '任丘市源创商贸有限公司',
 '任丘市雨田商贸有限公司',
 '优酪（滨州）商贸有限公司',
 '佛山市承佐贸易有限公司',
 '佰利工业设备（深圳）有限公司',
 '保定市新雨峰网络科技有限公司',
 '保定市方洲锟腾商贸有限公司',
 '信阳凯旋实业有限公司',
 '信阳市萱盛实业有限公司',
 '兰州乾圣商贸有限公司',
 '兰州动达商贸有限公司',
 '兰考县鑫源副食批发部',
 '兴化市东兴百货商贸有限公司',
 '内丘县王

In [132]:
len(name_checked_lst)

487

In [133]:
df['原本表格里面的文字'] = name_checked_lst

df

,公司名字,原本表格里面的文字,企查查搜索结果数,公司标签,法定法人,注册资本,成立日期,法人邮箱,法人电话,公司地址,公司状态,股东,网站
0,七台河市利丰源商贸有限公司,七台河利丰源商贸有限公司,1,,徐庆利,30万元人民币,2016-09-26,电话,\n 邮箱：\n ...,黑龙江省七台河市桃山区桃南街怡安小区7号楼12号车库,存续,徐庆利; 赵施梅;,https://www.qcc.com//firm/64EC397.shtml
1,三亚凤凰惠隆冷饮商行,三亚凤凰惠隆冷饮商行,1,,郭伟弟,-,2012-11-28,电话,\n 邮箱：\n ...,海南省三亚市天涯区芒果村,存续,陈南光;,https://www.qcc.com//firm/87TY0Z2.shtml
2,三明市合林贸易有限公司,三明合林贸易有限公司,4,,吴合林,50万元人民币,2019-11-20,电话,\n 邮箱：\n ...,福建省三明市三元区富文路11号6幢202、203室,存续,蔡琦; 吴合林;,https://www.qcc.com//firm/830GLN7.shtml
3,三河市俊鑫奕城商贸有限公司,三河市俊鑫奕城商贸有限公司,1,,李春梅,200万元人民币,2015-01-13,电话,\n 邮箱：\n ...,河北省廊坊市三河市燕郊高新区南外环路北侧金谷·爱舒荷第A10幢1单元14层1406号房,存续,李春梅; 李俊; 吴艳丽;,https://www.qcc.com//firm/69SUPSP.shtml
4,上海久耶供应链管理有限公司,上海久耶供应链管理有限公司,19,战略投资,张冰,333.7607万元人民币,2014-11-10,电话,\n 邮箱：\n ...,官网：\n www.jiuy...,存续,张冰; 峰尚达资本投资管理（深圳）有限公司; 张乐; 侯高阳; 上海郑明投资（集团）有限公司...,https://www.qcc.com//firm/94QA4S3.shtml
...,...,...,...,...,...,...,...,...,...,...,...,...,...
482,宝鸡尚优千品商贸有限公司,鸡尚优千品商贸有限公司,1,,尚斌,50万元人民币,2018-12-06,电话,\n 邮箱：\n ...,陕西省宝鸡市渭滨区相家庄101号九华龙石小区东侧库房,在业,徐旭; 尚斌;,https://www.qcc.com//firm/56UY7D2.shtml
483,鸡西市易和商贸有限公司,鸡西市易和商贸有限公司,1,,李凤莲,10万元人民币,2015-01-16,电话,\n 邮箱：\n ...,黑龙江省鸡西市鸡冠区东山阳光家园11号楼1单元103,存续,李凤莲; 李雅斌;,https://www.qcc.com//firm/62HNCQR.shtml
484,鹤岗市悦生活商贸有限公司,鹤岗市悦生活商贸有限公司,1,,裴子同,30万元人民币,2016-11-14,电话,\n 邮箱：\n ...,鹤岗市南山区33委交警队2号住宅楼-105室(车库),存续,裴子同; 杨洪振;,https://www.qcc.com//firm/29K95KO.shtml
485,齐河县旭泽商贸有限公司,齐河县旭泽商贸有限公司,1,,朱芬芬,60万元人民币,2018-03-01,电话,\n 邮箱：\n ...,山东省德州市齐河县城区齐贸大街392号2排4号,在业,朱芬芬; 张正婷;,https://www.qcc.com//firm/35HGRAZ.shtml


In [135]:
check_lst = []

# for i in range (3):

for i in range (df.shape[0]):
    
    gd_temp = df.loc[i,"股东"]
    if (gd_temp != gd_temp):
        df.at[i,"股东"] = df.loc[i,"法定法人"] + "; "    

df

,公司名字,原本表格里面的文字,企查查搜索结果数,公司标签,法定法人,注册资本,成立日期,法人邮箱,法人电话,公司地址,公司状态,股东,网站
0,七台河市利丰源商贸有限公司,七台河利丰源商贸有限公司,1,,徐庆利,30万元人民币,2016-09-26,电话,\n 邮箱：\n ...,黑龙江省七台河市桃山区桃南街怡安小区7号楼12号车库,存续,徐庆利; 赵施梅;,https://www.qcc.com//firm/64EC397.shtml
1,三亚凤凰惠隆冷饮商行,三亚凤凰惠隆冷饮商行,1,,郭伟弟,-,2012-11-28,电话,\n 邮箱：\n ...,海南省三亚市天涯区芒果村,存续,陈南光;,https://www.qcc.com//firm/87TY0Z2.shtml
2,三明市合林贸易有限公司,三明合林贸易有限公司,4,,吴合林,50万元人民币,2019-11-20,电话,\n 邮箱：\n ...,福建省三明市三元区富文路11号6幢202、203室,存续,蔡琦; 吴合林;,https://www.qcc.com//firm/830GLN7.shtml
3,三河市俊鑫奕城商贸有限公司,三河市俊鑫奕城商贸有限公司,1,,李春梅,200万元人民币,2015-01-13,电话,\n 邮箱：\n ...,河北省廊坊市三河市燕郊高新区南外环路北侧金谷·爱舒荷第A10幢1单元14层1406号房,存续,李春梅; 李俊; 吴艳丽;,https://www.qcc.com//firm/69SUPSP.shtml
4,上海久耶供应链管理有限公司,上海久耶供应链管理有限公司,19,战略投资,张冰,333.7607万元人民币,2014-11-10,电话,\n 邮箱：\n ...,官网：\n www.jiuy...,存续,张冰; 峰尚达资本投资管理（深圳）有限公司; 张乐; 侯高阳; 上海郑明投资（集团）有限公司...,https://www.qcc.com//firm/94QA4S3.shtml
...,...,...,...,...,...,...,...,...,...,...,...,...,...
482,宝鸡尚优千品商贸有限公司,鸡尚优千品商贸有限公司,1,,尚斌,50万元人民币,2018-12-06,电话,\n 邮箱：\n ...,陕西省宝鸡市渭滨区相家庄101号九华龙石小区东侧库房,在业,徐旭; 尚斌;,https://www.qcc.com//firm/56UY7D2.shtml
483,鸡西市易和商贸有限公司,鸡西市易和商贸有限公司,1,,李凤莲,10万元人民币,2015-01-16,电话,\n 邮箱：\n ...,黑龙江省鸡西市鸡冠区东山阳光家园11号楼1单元103,存续,李凤莲; 李雅斌;,https://www.qcc.com//firm/62HNCQR.shtml
484,鹤岗市悦生活商贸有限公司,鹤岗市悦生活商贸有限公司,1,,裴子同,30万元人民币,2016-11-14,电话,\n 邮箱：\n ...,鹤岗市南山区33委交警队2号住宅楼-105室(车库),存续,裴子同; 杨洪振;,https://www.qcc.com//firm/29K95KO.shtml
485,齐河县旭泽商贸有限公司,齐河县旭泽商贸有限公司,1,,朱芬芬,60万元人民币,2018-03-01,电话,\n 邮箱：\n ...,山东省德州市齐河县城区齐贸大街392号2排4号,在业,朱芬芬; 张正婷;,https://www.qcc.com//firm/35HGRAZ.shtml


In [136]:
''.join(gd)

gd_sep = ''.join(gd).split("; ")




In [144]:
gd_sep

df_full = pd.read_excel ('data/经销商+交易对手方最全的.xlsx')

df_full


,Client,Type,Problem,人名,Problem: 1 大公司或个人；2是之前名字比较模糊，后面通过企查查查找确认的；3是经销商和对手方重合的；4是之前名字比较模糊，后面通过企查查查不到的
0,个人客户,经销商,1.0,NaN,其他问题
1,乌鲁木齐王宗英,经销商,1.0,王宗英,1. 个人客户，员工因为没有具体信息就没有看
2,京东商城,经销商,1.0,NaN,2. 大公司，根据Sara的判断，我也没有用企查查
3,员工,对手方,1.0,NaN,NaN
4,员工,对手方,1.0,NaN,NaN
...,...,...,...,...,...
607,鸡尚优千品商贸有限公司,对手方,NaN,NaN,NaN
608,鸡西市易和商贸有限公司,经销商,NaN,NaN,NaN
609,鹤岗市悦生活商贸有限公司,经销商,NaN,NaN,NaN
610,齐河县旭泽商贸有限公司,经销商,NaN,NaN,NaN


In [149]:
for item in (df_full.人名.dropna()):
    gd_sep.append(item)


In [150]:
name_occur = [[x,gd_sep.count(x)] for x in set(gd_sep)]

name_occur

[['', 1],
 ['陈淑英', 1],
 ['卓书鸿', 1],
 ['罗秀印', 1],
 ['朱小敏', 1],
 ['冯英', 1],
 ['王兆敏', 1],
 ['秦阳丽', 1],
 ['陈云鹏', 1],
 ['朱绍范', 1],
 ['房美菊', 1],
 ['刘海涛', 1],
 ['段文祥', 1],
 ['张飞', 1],
 ['左进', 1],
 ['周玉花', 1],
 ['李学识', 2],
 ['刘秀英', 1],
 ['李娜', 2],
 ['厦门建发仓储有限公司', 1],
 ['张法伟', 1],
 ['李跃', 1],
 ['孙丽琼', 1],
 ['藏强', 1],
 ['张莉娟', 1],
 ['袁迪', 1],
 ['周友新', 1],
 ['龚群', 1],
 ['江苏通湖速运有限公司', 1],
 ['陈春柱', 1],
 ['高丽俊', 1],
 ['王播', 1],
 ['沈阳快行天下物流供应链管理有限公司', 1],
 ['王龙丰', 1],
 ['胡英', 1],
 ['张银龙', 1],
 ['郑永达', 1],
 ['李强', 1],
 ['谢红英', 1],
 ['王辉', 1],
 ['高志慧', 1],
 ['唐琬滢', 1],
 ['宁波远吉鼎泰投资管理合伙企业（有限合伙）', 1],
 ['商玉敏', 1],
 ['丁正平', 1],
 ['李星杰', 1],
 ['吴建超', 3],
 ['\n                                        河北信产新材创业投资有限公司', 1],
 ['李永红', 1],
 ['王丹丹', 1],
 ['云南九曳供应链管理有限公司', 1],
 ['李宗涛', 1],
 ['张会祥', 1],
 ['范本龙', 1],
 ['XIN RONG HUANG', 1],
 ['王宝君', 1],
 ['孔祥艳', 1],
 ['闵学东', 1],
 ['高伟霞', 1],
 ['李春萍', 1],
 ['沈宝山', 1],
 ['王贵宝', 1],
 ['姬绪霞', 1],
 ['吴亚杰', 1],
 ['张喜贤', 1],
 ['郑军', 2],
 ['佟秀俊', 1],
 ['刘志东', 2],
 ['王月兵', 1],


In [151]:
df_out = pd.DataFrame(data = name_occur, columns = ['name', 'times'])

df_out.to_csv("各家公司股东高管出现次数.csv")

df_out

,name,times
0,,1
1,陈淑英,1
2,卓书鸿,1
3,罗秀印,1
4,朱小敏,1
...,...,...
1274,高永青,1
1275,冯波,1
1276,吴艳丽,1
1277,利群商业集团股份有限公司,2


In [152]:
df_strange = df_out[df_out['times']>1]

df_strange

,name,times
16,李学识,2
18,李娜,2
46,吴建超,3
65,郑军,2
67,刘志东,2
...,...,...
1206,张志强,2
1213,刘波,2
1216,刘志松,2
1222,张毅,2


In [153]:
check_lst = []

# for i in range (3):

for i in range (df.shape[0]):
    
    gd_temp = df.loc[i,"股东"]
    
    if not gd_temp:
        gd_temp = df.loc[i,"法定法人"] + "; "
    
    lst = gd_temp.split("; ")
        
    temp = ""
    for t in lst:
        
        if t in df_strange.name.tolist():
            temp += (t + "-" + str(int(df_strange.loc[(df_strange[df_strange['name']==t].index.values), 'times']))+"; ")
    
    check_lst.append(temp)
    
check_lst

['',
 '',
 '',
 '李俊-2; ',
 '刘波-2; ',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '李俊-2; ',
 '',
 '',
 '',
 '王万祥-2; 刘志东-2; ',
 '',
 '',
 '',
 '',
 '张尚华-2; ',
 '',
 '冯文燕-2; ',
 '',
 '',
 '刘志松-2; 蒋红红-2; ',
 '',
 '陈宏-2; 吕利-2; ',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '王文-3; ',
 '董小翠-2; ',
 '',
 '',
 '刘冬梅-2; ',
 '蔡连青-2; ',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '李文杰-2; ',
 '李文杰-2; ',
 '',
 '',
 '',
 '',
 '',
 '李亚峰-2; 杨爽利-2; ',
 '李金山-2; 付鑫-2; ',
 '',
 '李金山-2; 付鑫-2; ',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '杨柳-2; ',
 '',
 '贾培旭-2; 王洋-3; 罗桂江-2; ',
 '鹿菊荣-2; 杨晓燕-2; 杨慧娟-2; ',
 '杨晓燕-2; 杨慧娟-2; 鹿菊荣-2; ',
 '秦国贞-2; ',
 '',
 '',
 '张毅-2; ',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '崔天博-2; ',
 '尚卫华-2; ',
 '',
 '',
 '',
 '郑军-2; 刘德华-4; ',
 '',
 '',
 '',
 '张勇-3; ',
 '',
 '',
 '',
 '吴霖-2; ',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '秦国贞-2; 李亚峰-2; 杨爽利-2; ',
 '',
 '',
 '',
 '',
 '',
 '张勇-3; ',
 '',
 '',
 '',
 '杜庆建-2; ',
 '',
 '',
 '王春峰-2; ',
 '王春峰-2; ',
 '',
 '',
 '',
 '',
 '苏文文-2; ',
 '苏文文-2

In [154]:
df['有重复的'] = check_lst

df

,公司名字,原本表格里面的文字,企查查搜索结果数,公司标签,法定法人,注册资本,成立日期,法人邮箱,法人电话,公司地址,公司状态,股东,网站,有重复的
0,七台河市利丰源商贸有限公司,七台河利丰源商贸有限公司,1,,徐庆利,30万元人民币,2016-09-26,电话,\n 邮箱：\n ...,黑龙江省七台河市桃山区桃南街怡安小区7号楼12号车库,存续,徐庆利; 赵施梅;,https://www.qcc.com//firm/64EC397.shtml,
1,三亚凤凰惠隆冷饮商行,三亚凤凰惠隆冷饮商行,1,,郭伟弟,-,2012-11-28,电话,\n 邮箱：\n ...,海南省三亚市天涯区芒果村,存续,陈南光;,https://www.qcc.com//firm/87TY0Z2.shtml,
2,三明市合林贸易有限公司,三明合林贸易有限公司,4,,吴合林,50万元人民币,2019-11-20,电话,\n 邮箱：\n ...,福建省三明市三元区富文路11号6幢202、203室,存续,蔡琦; 吴合林;,https://www.qcc.com//firm/830GLN7.shtml,
3,三河市俊鑫奕城商贸有限公司,三河市俊鑫奕城商贸有限公司,1,,李春梅,200万元人民币,2015-01-13,电话,\n 邮箱：\n ...,河北省廊坊市三河市燕郊高新区南外环路北侧金谷·爱舒荷第A10幢1单元14层1406号房,存续,李春梅; 李俊; 吴艳丽;,https://www.qcc.com//firm/69SUPSP.shtml,李俊-2;
4,上海久耶供应链管理有限公司,上海久耶供应链管理有限公司,19,战略投资,张冰,333.7607万元人民币,2014-11-10,电话,\n 邮箱：\n ...,官网：\n www.jiuy...,存续,张冰; 峰尚达资本投资管理（深圳）有限公司; 张乐; 侯高阳; 上海郑明投资（集团）有限公司...,https://www.qcc.com//firm/94QA4S3.shtml,刘波-2;
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
482,宝鸡尚优千品商贸有限公司,鸡尚优千品商贸有限公司,1,,尚斌,50万元人民币,2018-12-06,电话,\n 邮箱：\n ...,陕西省宝鸡市渭滨区相家庄101号九华龙石小区东侧库房,在业,徐旭; 尚斌;,https://www.qcc.com//firm/56UY7D2.shtml,徐旭-2; 尚斌-2;
483,鸡西市易和商贸有限公司,鸡西市易和商贸有限公司,1,,李凤莲,10万元人民币,2015-01-16,电话,\n 邮箱：\n ...,黑龙江省鸡西市鸡冠区东山阳光家园11号楼1单元103,存续,李凤莲; 李雅斌;,https://www.qcc.com//firm/62HNCQR.shtml,
484,鹤岗市悦生活商贸有限公司,鹤岗市悦生活商贸有限公司,1,,裴子同,30万元人民币,2016-11-14,电话,\n 邮箱：\n ...,鹤岗市南山区33委交警队2号住宅楼-105室(车库),存续,裴子同; 杨洪振;,https://www.qcc.com//firm/29K95KO.shtml,
485,齐河县旭泽商贸有限公司,齐河县旭泽商贸有限公司,1,,朱芬芬,60万元人民币,2018-03-01,电话,\n 邮箱：\n ...,山东省德州市齐河县城区齐贸大街392号2排4号,在业,朱芬芬; 张正婷;,https://www.qcc.com//firm/35HGRAZ.shtml,


In [197]:
jingxiao = pd.read_excel('data/经销商full list-截止2020年5月底.xlsx')

duishou = pd.read_excel('data/银行存款-交易对手方.xlsx')

duishou_lst = duishou.Client.tolist()

jingxiao_lst = jingxiao.客户简称.tolist()

if_in_duishou = []

if_in_jingxiao = []

In [198]:

for i in range (df.shape[0]):
    if_in_duishou.append(1*(df.loc[i,"原本表格里面的文字"] in duishou_lst))
    if_in_jingxiao.append(1*(df.loc[i,"原本表格里面的文字"] in jingxiao_lst))

            
df['对手'] = if_in_duishou
df['经销商'] = if_in_jingxiao


In [199]:
sum(if_in_jingxiao)

415

In [200]:
df.to_excel("妙飞_供应商经销商_重叠人员分析_V2.xlsx")